In [ ]:
#default_exp datablock

# DataBlock

> process timeseries datasets with DataBloc

In [ ]:
import tsai
from tsai.all import *

In [ ]:
print('tsai       :', tsai.__version__)
print('fastai2    :', fastai2.__version__)
print('fastcore   :', fastcore.__version__)
print('torch      :', torch.__version__)
print('scipy      :', sp.__version__)
print('numpy      :', np.__version__)
print('pandas     :', pd.__version__)
print(f'Total RAM  : {bytes2GB(psutil.virtual_memory().total):5.2f} GB')
print(f'Used RAM   : {bytes2GB(psutil.virtual_memory().used):5.2f} GB')
print('n_cpus     :', cpus)
iscuda = torch.cuda.is_available()
if iscuda: print('device     : {} ({})'.format(device, torch.cuda.get_device_name(0)))
else: print('device     :', device)

tsai       : 0.1.0
fastai2    : 0.0.18
fastcore   : 0.1.18
torch      : 1.3.1
scipy      : 1.4.1
numpy      : 1.18.1
pandas     : 0.25.3
Total RAM  : 15.56 GB
Used RAM   :  6.74 GB
n_cpus     : 4
device     : cuda (GeForce GTX 950)


In [ ]:
#export
from torchtools.data import *
from torchtools.datasets import *

In [ ]:
# from torchtools.augmentations import *

In [ ]:
# from fastai2.data.all import *

In [ ]:
import pandas as pd
import numpy as np
from functools import partial

In [ ]:
# from fastai2.vision.all import *

In [ ]:
_verbose=True

### Until now 

In [ ]:
df_main = pd.read_csv('./data/custom/bi_sample_anon.csv', nrows=200000)

In [ ]:
## simple config
x_cols = [[f'x{i}_{j}' for j in range(10)] for i in range(6)]
dep = 'y0'
n_train = 8000

items is a list of (x,y) tuples  
methods called when passing items to DataBlock:
- DataBlock.datasets(items, ....)
- Dataset(items, ...)
- TfmdList(items, ...)
- L(items, ...)
- CollBase.__init__(items, ...)



In [ ]:
# def get_items(df):
#     return list(map(list,df_to_items(df_main, x_cols, dep, n_train)[0]))

# def get_x(x):
#     pv(f'get_x item {x}', verbose=_verbose)
#     return x[0]
# def get_y(x):
#     pv(f'get_y item {x}', verbose=_verbose)
#     return x[1]

# items = get_items(df_main)

### TSAI Excursion

In [ ]:
items_0 = df_to_items(df_main, x_cols, dep, n_train)[0]

(200000, 6, 10)


In [ ]:
def _items_to_arrays(items):
    return np.stack([x[0] for x in items]), np.stack([x[1] for x in items])

In [ ]:
X,y = _items_to_arrays(items_0)

In [ ]:
X.shape, y.shape

((200000, 6, 10), (200000,))

In [ ]:
splits = [L(range(160000)), L(range(160000,185000))]
# splits = [L(range(1600)), L(range(1600,1850))]

tfms  = [None, None]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits, inplace=True)

In [ ]:
@Transform
def ToTT(x:tsai.data.core.TSTensor):
    print('ToTT')
    return cast(x, TSTensor)

ToTT.order = 150
ToTT.split_idx = 0

In [ ]:
# dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 512], num_workers=0)
dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 512], num_workers=0, 
#                                after_batch=[TSNormalize(by_var=False)
                               batch_tfms=[TSNormalize(by_sample=False, by_var=True), ToTT])

In [ ]:
dls[1].one_batch()

(TSTensor(samples:250, vars:6, len:10),
 tensor([ 100.0000,  -43.2900,  100.0000,  -40.3226,  100.0000,  -30.9598,
          100.0000,  -15.7480,  100.0000,  -33.2226,  -28.9017,  100.0000,
         -112.3596,  100.0000,  -64.1026,  -47.6190,  -94.3396,  -29.1545,
          -47.3934,  -69.9301,  100.0000,  -58.8235,  -71.4286,  -45.8716,
          100.0000,  -12.7389,  100.0000,  -21.5517,  -33.8983,  100.0000,
          100.0000,  -12.1359,  -59.1716,  -57.1429,  -46.7290,  -72.9927,
          100.0000,  -76.3359,  100.0000,  -24.9377,  -78.7402,  -35.2113,
          -64.9351,  -45.4545, -178.5714,  -13.6240,  -48.3092,  -59.8802,
          -67.5676,  -43.2900,  100.0000,  -55.8659,  -53.4759,  100.0000,
         -123.4568,  -24.8139,  -37.0370,  100.0000,  100.0000,  -25.6410,
          100.0000,  -34.4828, -103.0928,  -27.0270,  -26.5957,  100.0000,
          -69.4444,  100.0000,  -80.0000,  -42.7350,  100.0000,   -4.7619,
          -75.1880,  100.0000,  -74.0741,  -42.7350, -185.18

In [ ]:
tfms = [TSCrop, TSTimeNoise, TSDenoise]

In [ ]:
for tfm in tfms: tfm.split_idx=0

In [ ]:
Pipeline(noop).fs[0]

noop: (object,object) -> noop 

In [ ]:
# #export
# _batch_tfms = ('after_item','before_batch','after_batch')

# class NumpyDataLoaders(DataLoaders):
#     _xblock = NumpyTensorBlock
#     _dl_type = NumpyDataLoader 
#     def __init__(self, *loaders, path='.', device=default_device()):
#         self.loaders,self.path = list(loaders),Path(path)
#         self.device = device
        
#     @classmethod
#     @delegates(DataLoaders.from_dblock)
#     def from_numpy(cls, X, y=None, splitter=None, valid_pct=0.2, seed=0, item_tfms=None, batch_tfms=None, **kwargs):
#         "Create timeseries dataloaders from arrays (X and y, unless unlabeled)"
#         if splitter is None: splitter = RandomSplitter(valid_pct=valid_pct, seed=seed)
#         getters = [ItemGetter(0), ItemGetter(1)] if y is not None else [ItemGetter(0)]
#         dblock = DataBlock(blocks=(cls._xblock, CategoryBlock),
#                            getters=getters,
#                            splitter=splitter,
#                            item_tfms=item_tfms,
#                            batch_tfms=batch_tfms)

#         source = itemify(X) if y is None else itemify(X,y)
#         return cls.from_dblock(dblock, source, **kwargs)

In [ ]:
from torchtools.augmentations import *

In [ ]:
# dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 512], num_workers=0)
dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 512], num_workers=0, 
                              after_batch=Pipeline(noop),
                            batch_tfms=[TSNormalize(by_sample=False, by_var=True), ToTT, 
                                           TimestepZero(split_idx=0, verbose=True)])

In [ ]:
dls[1].one_batch()

(TSTensor(samples:250, vars:6, len:10),
 tensor([ 100.0000,  -43.2900,  100.0000,  -40.3226,  100.0000,  -30.9598,
          100.0000,  -15.7480,  100.0000,  -33.2226,  -28.9017,  100.0000,
         -112.3596,  100.0000,  -64.1026,  -47.6190,  -94.3396,  -29.1545,
          -47.3934,  -69.9301,  100.0000,  -58.8235,  -71.4286,  -45.8716,
          100.0000,  -12.7389,  100.0000,  -21.5517,  -33.8983,  100.0000,
          100.0000,  -12.1359,  -59.1716,  -57.1429,  -46.7290,  -72.9927,
          100.0000,  -76.3359,  100.0000,  -24.9377,  -78.7402,  -35.2113,
          -64.9351,  -45.4545, -178.5714,  -13.6240,  -48.3092,  -59.8802,
          -67.5676,  -43.2900,  100.0000,  -55.8659,  -53.4759,  100.0000,
         -123.4568,  -24.8139,  -37.0370,  100.0000,  100.0000,  -25.6410,
          100.0000,  -34.4828, -103.0928,  -27.0270,  -26.5957,  100.0000,
          -69.4444,  100.0000,  -80.0000,  -42.7350,  100.0000,   -4.7619,
          -75.1880,  100.0000,  -74.0741,  -42.7350, -185.18

In [ ]:
learn_small = Learner(model=model, loss_func=loss_fn, dls=dls, metrics=unweighted_profit)

In [ ]:
learn_small.fit_one_cycle(5)

epoch,train_loss,valid_loss,unweighted_profit,time
0,-1.815828,0.191247,-2.856286,00:00
1,-3.272676,0.045588,-2.856286,00:00
2,-6.516383,-0.710317,1.295607,00:00
3,-9.294381,-0.642300,0.793727,00:00
4,-11.678082,-0.391897,1.466111,00:00


ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero


In [ ]:
preds = learn_small.get_preds(1)

In [ ]:
unweighted_profit(*preds)

tensor(1.4661)

In [ ]:
tfms = all_erasing_augs(magnitude=0.4) + all_noise_augs(magnitude=0.4) + all_zoom_augs(magnitude=0.4)

In [ ]:
splits = [L(range(160000)), L(range(160000,185000))]
# splits = [L(range(1600)), L(range(1600,1850))]

tfms  = [None, None]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits, inplace=True)

# dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 512], num_workers=0)
dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 256], num_workers=0, 
#                                after_batch=[TSNormalize(by_var=False)
                               batch_tfms=[TSNormalize(by_sample=False, by_var=True), ToTT, RandAugment(N=3, magnitude=0.4, tfms=tfms)
                                           ])

In [ ]:
xb, yb = dls[0].one_batch()

ToTT
[0 1 2 3 4 5 6 7]


In [ ]:
dls.after_batch.tfms[3].magnitude

0.4

In [ ]:
# dls.show_batch()

In [ ]:
from torchtools.models import *
from torchtools.core import *

In [ ]:
## debug
class InceptionBlock(nn.Module):
    def __init__(self,c_in,bottleneck=32,ks=40,nb_filters=32,residual=True,depth=6):

        super().__init__()

        self.residual = residual
        self.depth = depth

        #inception & residual layers
        inc_mods = []
        res_layers = []
        res = 0
        for d in range(depth):
            inc_mods.append(
                Inception(c_in if d == 0 else nb_filters * 4, bottleneck=bottleneck if d > 0 else 0,ks=ks,
                          nb_filters=nb_filters))
            if self.residual and d % 3 == 2:
                res_layers.append(shortcut(c_in if res == 0 else nb_filters * 4, nb_filters * 4))
                res += 1
            else: res_layer = res_layers.append(None)
        self.inc_mods = nn.ModuleList(inc_mods)
        self.res_layers = nn.ModuleList(res_layers)
        self.act = nn.ReLU()
        
    def forward(self, x):
#         print(f'block dtype {x.dtype}')
        res = x
        for d, l in enumerate(range(self.depth)):
            x = self.inc_mods[d](x)
            if self.residual and d % 3 == 2:
                res = self.res_layers[d](res)
                x += res
                res = x
                x = self.act(x)
        return x


class InceptionTime(nn.Module):
    def __init__(self,c_in,c_out,bottleneck=32,ks=40,nb_filters=32,residual=True,depth=6):
        super().__init__()
        self.block = InceptionBlock(c_in,bottleneck=bottleneck,ks=ks,nb_filters=nb_filters,
                                    residual=residual,depth=depth)
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(nb_filters * 4, c_out)

    def forward(self, x):
#         print(x.dtype)
#         x.to(torch.float)
        x = self.block(x)
#         print(x.shape)
        x = self.gap(x).squeeze(-1)
        x = self.fc(x)
        return x


class InceptionTimeSgm(nn.Module):
    '''
    add a sigmoid layer to InceptionTime to get the ouput in a certain range
    '''
    
    def __init__(self, n_in, n_out):
        super().__init__()
        self.mod = nn.Sequential(InceptionTime(n_in, n_out), Sigmoid(-1., 1.))
        
    def forward(self, x):
        x = x.float()
#         print(f'InceptionTimeSgm dtype {x.dtype}')
        return self.mod(x)

In [ ]:
xb, yb = dls[1].one_batch()
unweighted_profit(learn.model(xb)[0].squeeze(),yb)

tensor(-2.8563, device='cuda:0')

In [ ]:
#export
class ResNetSgm(nn.Module):
    def __init__(self,c_in, c_out):
        super().__init__()
        nf = 64

        self.block1 = ResBlock(c_in, nf, ks=[7, 5, 3], act_fn='relu')
        self.block2 = ResBlock(nf, nf * 2, ks=[7, 5, 3], act_fn='relu')
        self.block3 = ResBlock(nf * 2, nf * 2, ks=[7, 5, 3], act_fn='relu')
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(nf * 2, c_out)
        self.sig = SigmoidRange(-1,1)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.gap(x).squeeze(-1)
        x = self.fc(x)
        return self.sig(x)

In [ ]:
#cuda
loss_fn = get_loss_fn('leaky_loss', alpha=0.5)

In [ ]:
splits = [L(range(160000)), L(range(160000,185000))]
# splits = [L(range(1600)), L(range(1600,1850))]

tfms  = [None, None]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits, inplace=True)

In [ ]:
ra_tfms = all_erasing_augs(magnitude=0.4) + all_noise_augs(magnitude=0.4) + all_zoom_augs(magnitude=0.4)
# dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 512], num_workers=0)
dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 256], num_workers=0, 
#                                after_batch=[TSNormalize(by_var=False)
                               batch_tfms=[TSNormalize(by_sample=False, by_var=True), ToTT, 
                                           RandAugment(N=3, magnitude=0.4, tfms=ra_tfms)
                                           ])

tfms [Dimout: (TSTensor,object) -> encodes , Cutout: (TSTensor,object) -> encodes , TimestepZero: (TSTensor,object) -> encodes , Crop: (TSTensor,object) -> encodes , RandomCrop: (TSTensor,object) -> encodes , Maskout: (TSTensor,object) -> encodes , YWarp: (TSTensor,object) -> encodes , YNormal: (TSTensor,object) -> encodes , YScale: (TSTensor,object) -> encodes , TimeWarp: (TSTensor,object) -> encodes , TimeNormal: (TSTensor,object) -> encodes , Zoomin: (TSTensor,object) -> encodes , Zoomout: (TSTensor,object) -> encodes , RandZoom: (TSTensor,object) -> encodes , RandTimesteps: (TSTensor,object) -> encodes ]


In [ ]:
set_seed(1234)

In [ ]:
## for tsai
model = InceptionTimeSgm(6,1)
learn = Learner(dls, model, loss_fn, metrics=[unweighted_profit, partial(unweighted_profit, threshold=0.5)])

In [ ]:
learn.fit_one_cycle(20, lr_max=1e-5, wd=0.03)

epoch,train_loss,valid_loss,unweighted_profit,unweighted_profit,time
0,-0.072998,-0.027973,-0.067943,0.000000,00:38
1,-0.003616,-0.058803,0.154930,0.000000,00:37
2,-0.152651,-0.096016,-0.021723,0.000000,00:37
3,-0.126958,-0.165256,0.149439,0.000000,00:33
4,-0.054351,-0.252393,0.046710,0.000000,00:32
5,-0.212057,-0.269473,0.037011,0.003273,00:33
6,-0.417109,-0.322774,0.194249,0.007550,00:34
7,-0.323409,-0.282049,0.067071,0.002660,00:35
8,-0.273400,-0.411014,0.165297,0.013048,00:33
9,-0.507305,-0.410874,0.077715,0.019656,00:38


ToTT
[2 3 4 6 7 9]
ToTT
[1 2 3 4 5 6 7 8 9]
[3 4 5 6 7 8 9]
[5 6 7 8 9]
ToTT
[3 4 5 6 7 8]
ToTT
ToTT
[1 3 4 5 8 9]
[3 4 5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8]
ToTT
[0 2 3 4 7 9]
ToTT
[4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
[3 4 5 6 7 8 9]
ToTT
ToTT
[3 4 5 6 7]
ToTT
[2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
[3 4 5 6 7]
ToTT
ToTT
[0 1 3 5 7 9]
ToTT
ToTT
ToTT
ToTT
[1 2 3 5 7 8]
ToTT
[0 2 5 7 8 9]
ToTT
ToTT
ToTT
[1 2 5 6 7 8]
ToTT
ToTT
[5 6 7 8 9]
[5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8]
[0 3 4 5 7 8]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
ToTT
[5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
[0 1 3 6 8 9]
ToTT
ToTT
ToTT
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
[0 1 2 3 4 5 6 7]
ToTT
[2 4 5 6 7 9]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
ToTT
ToTT
[0 1 2 3 4 5 6 7 8]
[0 2 4 5 7 9]
ToTT
[2 3 4 5 6 7]
ToTT
[0 1 2 3

ToTT
[2 3 4 5 6 7 8 9]
ToTT
[3 4 5 6 7 8]
ToTT
[4 5 6 7 8 9]
ToTT
[1 2 3 4 5]
ToTT
[2 3 4 5 6 7]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
[2 3 4 5 6]
ToTT
[4 5 6 7 8]
[3 4 6 7 8 9]
ToTT
[5 6 7 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[5 6 7 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7]
[2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
[0 1 2 3 7 8]
[1 2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[0 2 3 6 8 9]
ToTT
ToTT
[1 3 4 6 7 8]
ToTT
[0 2 3 4 6 7]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7]
ToTT
[0 1 2 5 6 7]
ToTT
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[0 1 2 5 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
[1 2 5 7 8 9]
ToTT
[3 4 6 7 8 9]
[5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
[1 2 4 5 6 7]
ToTT
[1 2 3 4 5 6 7 8]
ToTT
ToTT
[5 6 7 8 9]
[3 4 5 6 7 8]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[0 3 4 6 7 9]
ToTT
[1 3 4 6 8 9]
[3 4 5 6 7]
ToTT
[2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
[5 6 7 8 9]
ToTT
[2 3 4 5 7 9]
[0 1 2 3 4]
ToTT
[1 2 3 4 5]
ToTT
ToTT
[5 6 7 8 9]
[0 2 3 5 6 8]
ToTT
[4 5 6 7 8 9]
ToTT

ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 9]
ToTT
[0 2 4 6 7 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
[0 1 2 4 6 8]
ToTT
[4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
[1 4 5 6 7 8]
ToTT
ToTT
ToTT
[0 3 4 6 7 8]
[1 2 3 4 5]
ToTT
ToTT
ToTT
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
[0 5 6 7 8 9]
ToTT
ToTT
[0 2 3 5 7 8]
[2 3 4 5 6 7 8 9]
ToTT
ToTT
[0 1 6 7 8 9]
[2 3 4 5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
[0 1 2 3 4 5]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 7]
[1 2 3 4 5 6]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[0 1 3 5 6 9]
ToTT
[3 4 5 6 7 8 9]
[0 1 3 4 5 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8]
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
[1 2 3 4 7 8]
ToTT
[1 4 6 7 8 9]
ToTT
ToTT
[3 4 5 6 7 8 9]
ToTT
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
[3 4 5 6 7 8 9]
[5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[0 3 5 6 7 9]
ToTT
[1 2 3 4 5 6 7 8 9]
[1 2 3 4 6 9]
ToTT
[1 2 4 6 7 9]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT

ToTT
[4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7]
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5]
[1 2 3 4 5 6 7 8 9]
ToTT
[0 2 3 4 6 7]
ToTT
ToTT
ToTT
[0 3 4 5 6 9]
ToTT
[4 5 6 7 8 9]
ToTT
[4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8]
ToTT
ToTT
[3 4 5 6 7 8 9]
ToTT
[0 1 2 4 6 9]
[3 4 5 6 7]
ToTT
[4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
[2 3 4 5 6 7 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8]
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
[1 2 3 4 5 6 7]
ToTT
[0 1 2 3 4 5]
ToTT
[0 4 5 6 7 9]
ToTT
[3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
[3 4 5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
[0 2 3 5 6 7]
ToTT
ToTT
[3 4 5 6 7]
[3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[5 6 7 8 9]
ToTT
[3 4 5 6 7 8]
ToTT
[0 1 4 5 6 8]
ToTT
[0 1 2 3 4 5 6 7 8]
[2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 6 9]
ToTT
[1 2 3 4 5 6 7 8 9]
[4 5 6 7 8 9]
ToTT
[1 2 3 4 8 9]
ToTT
[4 5 6 7 8 9]
ToTT
ToTT
ToTT
[0 1 2 3 6 7]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
[5 6 7 8 9

...

ToTT
[3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5]
ToTT
[0 2 3 4 5 7]
[0 1 2 3 4 5 6 7 8]
[4 5 6 7 8 9]
ToTT
[2 3 4 5 7 9]
ToTT
[0 1 2 3 4 5 6 7]
ToTT
[3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
ToTT
[1 2 3 4 5 6 7 8]
ToTT
[3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
ToTT
[4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
ToTT
ToTT
[0 1 2 5 7 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 4 6 7 9]
ToTT
ToTT
ToTT
[5 6 7 8 9]
ToTT
[0 3 5 6 7 8]
ToTT
ToTT
[3 4 5 6 7]
[3 4 5 6 7]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 5 6 8 9]
[3 4 5 6 7]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
[4 5 6 7 8 9]
ToTT
[0 1 2 3 7 9]
ToTT
[3 4 5 6 7 8 9]
[2 3 5 6 7 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[5 6 7 8 9]
[0 2 4 7 8 9]
ToTT
ToTT
[1 2 4 7 8 9]
ToTT
[0 3 4 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[3 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[0 1 3 6 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7 8]
[2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7]
ToTT
[5 6 7 8 9]
[1 2 3 4 5 6 7 

ToTT
[1 2 3 4 5 6]
ToTT
ToTT
ToTT
[4 5 6 7 8]
ToTT
[0 1 2 3 6 8]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6]
ToTT
ToTT
ToTT
[1 3 5 6 7 9]
ToTT
ToTT
[2 3 4 5 6 7 8]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[2 4 5 6 7 9]
[4 5 6 7 8 9]
ToTT
ToTT
ToTT
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
[0 1 4 6 8 9]
ToTT
[1 2 4 5 7 8]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[4 5 6 7 8 9]
ToTT
ToTT
[3 4 5 6 7]
ToTT
ToTT
ToTT
[3 4 5 6 7 8 9]
ToTT
[3 4 5 6 7 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8]
ToTT
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
[2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7]
ToTT
ToTT
[0 1 3 4 5 8]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[4 5 6 7 8 9]
[1 3 5 6 7 8]
ToTT
[0 1 2 4 5 9]
ToTT
[0 2 3 4 6 8]
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
[4 5 6 7 8 9]
ToTT
ToTT
ToTT
[0 1 2 3 4 5 6 7]
ToTT
[2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7 8]
To

ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[5 6 7 8 9]
ToTT
[1 2 3 4 5]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
[2 3 4 5 6 7]
ToTT
ToTT
[0 1 3 6 7 9]
ToTT
[0 1 2 3 4 5 6 7 8]
[1 2 3 4 5 6 7 8]
ToTT
[5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8]
ToTT
ToTT
ToTT
ToTT
[1 2 3 4 5 6 7 8]
[2 3 4 5 6 7 8 9]
ToTT
[0 1 2 5 6 7]
ToTT
[3 4 5 6 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7]
ToTT
[2 3 4 5 6 7 8 9]
[2 3 4 7 8 9]
ToTT
[0 1 3 5 6 9]
[5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
[1 2 3 5 6 9]
ToTT
[2 3 4 5 6 7]
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7]
ToTT
[5 6 7 8 9]
ToTT
ToTT
ToTT
[0 1 4 5 7 8]
[2 3 4 5 6]
ToTT
ToTT
ToTT
[1 2 3 4 5 6 7]
[2 3 4 5 6 7 8 9]
ToTT
[1 2 3 5 6 9]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
[3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
[5 6 7 8 9]
ToTT
ToTT
[2 4 5 6 8 9]
ToTT
[0 1 2 3 4 5 6 7 8]
[0 1 2 3 4 5 6 7 8]
ToTT
ToTT
ToTT
[0 1 3 6 8 9]
ToTT
ToTT
[0 3 4 5 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8]
ToTT
ToTT
[2 3 5 6 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT


ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
[2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8]
ToTT
ToTT
ToTT
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
ToTT
[0 1 2 3 4 5 6 7 8]
[1 2 3 4 5 6]
ToTT
[5 6 7 8 9]
[1 2 3 7 8 9]
ToTT
[2 3 4 5 6 7 8]
ToTT
[4 5 6 7 8 9]
ToTT
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7]
ToTT
[1 3 4 5 6 7]
ToTT
[0 1 5 7 8 9]
ToTT
[0 1 2 3 4 7]
[3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
[0 1 2 3 4]
ToTT
ToTT
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6]
[0 1 4 6 8 9]
ToTT
[2 4 5 6 7 9]
[1 2 3 4 5 6 7]
ToTT
[0 1 4 5 8 9]
[5 6 7 8 9]
ToTT
[5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
ToTT
[5 6 7 8 9]
ToTT
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
[1 2 3 4 5 6 7 8]
[3 4 5 6 7 8 9]
ToTT
[0 1 2 5 6 8]
ToTT
[3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7]
ToTT
ToTT
[2 3 4 5 6 7 8]
ToTT
[3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8]
ToTT
[5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[5 6 7 8 9]
ToTT
ToTT
[2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[3 4 5 6 7]
ToTT
[3 4 5 6 7 8 9]
[1 2 3 4 5 

ToTT
[4 5 6 7 8 9]
ToTT
ToTT
ToTT
[3 4 5 6 7 8]
ToTT
[1 2 3 4 5 6 7 8 9]
[1 2 3 4 5 6]
ToTT
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 4 5 6 7 9]
ToTT
[0 1 2 3 4 5 6]
ToTT
[0 1 2 4 5 6]
ToTT
ToTT
[0 1 2 6 7 9]
[0 1 2 3 4 5 6 7 8]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 5 8 9]
ToTT
[0 3 4 5 6 8]
ToTT
[3 4 5 6 7 8]
ToTT
[1 2 3 4 5 6 7]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[3 4 5 6 7 8]
[1 2 3 4 5 6 7]
ToTT
[1 2 3 4 5 6 7 8 9]
[2 3 4 5 6]
ToTT
[2 3 5 6 7 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
[3 4 5 6 8 9]
ToTT
[0 2 3 4 5 6]
ToTT
[4 5 6 7 8 9]
[2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[0 1 3 4 6 7]
ToTT
[5 6 7 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[5 6 7 8 9]
[5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[

In [ ]:
learn.fit_one_cycle(20, lr_max=1e-5, wd=0.03)

epoch,train_loss,valid_loss,unweighted_profit,unweighted_profit,time
0,-0.080210,0.003429,-0.295270,0.000000,00:39
1,-0.126654,-0.032117,-0.127947,0.000000,00:37
2,-0.153243,-0.136671,-0.008716,0.000000,00:35
3,-0.120966,-0.185432,0.005733,0.000000,00:43
4,-0.396824,-0.340714,-0.001029,0.000000,00:51
5,-0.271871,-0.358625,0.017879,0.000000,00:42
6,-0.618765,-0.455368,0.040671,0.000000,00:50
7,-0.436391,-0.462714,-0.010640,0.000000,00:40
8,-0.537534,-0.516880,0.000559,0.000000,00:34
9,-0.508221,-0.538972,0.008900,0.004000,00:33


ToTT
[4 5 6 7 8 9]
[3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[0 1 3 6 8 9]
ToTT
ToTT
ToTT
ToTT
[0 1 2 3 6 8]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
[3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7 8]
[2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 6]
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
ToTT
ToTT
[1 2 3 4 5 6 7 8]
ToTT
[1 2 3 4 5 6 7 8 9]
[4 5 6 7 8 9]
ToTT
ToTT
[5 6 7 8 9]
ToTT
[5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7 8]
[4 5 6 7 8 9]
ToTT
[1 2 3 5 7 9]
[5 6 7 8 9]
ToTT
[0 2 3 6 7 8]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
[1 2 3 4 5]
ToTT
[3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8]
ToTT
ToTT
[3 4 5 6 7]
ToTT
ToTT
[3 4 5 6 7 8 9]
[1 2 4 5 6 8]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 8]
ToTT
[1 2 3 4 5 6 7]
ToTT
[1 3 5 6 7 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[0 2 3 4 7 9]
[2 3 4 5 6 7 8 9]
ToTT
[0 1 3 5 8 9]
ToTT
ToTT
[0 1 2 4 5 6]
ToTT
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
[4 5 6 7 8 9]
ToTT
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8 

ToTT
ToTT
[4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[4 5 6 7 8 9]
ToTT
[1 2 4 5 6 9]
ToTT
ToTT
ToTT
ToTT
[3 4 5 6 7]
[0 1 2 3 4 7]
[2 3 4 5 6 7]
ToTT
ToTT
[5 6 7 8 9]
[0 1 2 3 4 5 6 7]
ToTT
[2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[4 5 6 7 8 9]
[0 1 2 3 4 5]
ToTT
[1 2 3 4 5 6 7]
ToTT
[0 1 2 3 4 5 6 7 8]
[2 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 6 7]
ToTT
ToTT
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
ToTT
ToTT
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
[0 2 3 5 8 9]
ToTT
[4 5 6 7 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
[2 3 4 5 6]
[1 2 3 4 5 6 7 8]
ToTT
ToTT
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
ToTT
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
[4 5 6 7 8]
ToTT
[0 1 2 3 4 5 6 7]
ToTT
[3 4 5 6 7 8 9]
ToTT
ToTT
[0 1 2 3 4 5 6 7]
[4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
[2 3 4 5 6 7 8 9]
ToTT
ToTT
[1 

ToTT
[0 1 2 3 4 5]
[0 2 4 7 8 9]
ToTT
[1 2 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8]
ToTT
[2 3 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[4 5 6 7 8 9]
ToTT
[5 6 7 8 9]
ToTT
[1 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7 8]
[1 2 3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8]
ToTT
[2 3 4 5 6]
[1 2 3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
[5 6 7 8 9]
[3 4 5 6 7]
ToTT
[2 3 4 5 6 7 8]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8]
ToTT
[1 2 3 4 5 6 7 8 9]
[2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7 8]
[1 2 3 4 5 6 7 8]
ToTT
ToTT
ToTT
[4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5]
[0 1 2 3 6 8]
ToTT
[0 5 6 7 8 9]
[0 1 2 3 4]
[1 2 3 4 5 6 7 8 9]
ToTT
[5 6 7 8 9]
[4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
[1 2 3 4 5 6]
ToTT
[3 4 5 6 7 8 9]
[1 2 3 4 6 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 2 4 6 9]
ToTT
[0 1 2 3 4]
[2 3 4 5 6 7 8 9]
ToTT
[5 6 7 8 9]
[3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
[4 5 6 7 8]
ToTT
ToTT
ToTT
[4 5 6 7 8 9]

ToTT
[2 3 4 5 6 7 8 9]
ToTT
[0 1 4 5 6 7]
[2 3 4 5 6 7 8 9]
ToTT
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
[2 3 4 5 6]
[1 2 3 4 5]
ToTT
ToTT
[2 3 4 5 6 7 8 9]
[4 5 6 7 8 9]
ToTT
ToTT
ToTT
[1 2 3 4 5 6]
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[1 3 4 5 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 4 6 8 9]
[4 5 6 7 8 9]
ToTT
[4 5 6 7 8 9]
[1 3 5 6 7 8]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[2 3 4 5 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 4 5 8 9]
ToTT
ToTT
ToTT
ToTT
ToTT
[0 3 6 7 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
[3 4 5 6 7]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
ToTT
[3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[2 3 4 5 6 7 8]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[0 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
[4 5 6 7 8 9]
ToTT
ToTT
ToTT
ToTT
[0 2 5 6 7 8]
ToTT
[1 2 3 4 5 6 7 8 9]
[1 3 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8]
ToTT
[0 1 2 4 5 7]
ToTT
[3 4 5 6 7 8 9]
ToTT
ToTT
[5 6 7 8 9]
ToTT
[5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
[1 2 

...

ToTT
[3 4 5 6 7 8 9]
[0 2 5 6 7 8]
ToTT
[1 2 3 6 7 9]
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
ToTT
[3 4 5 6 7 8]
[0 2 3 5 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 3 6 7 8]
ToTT
ToTT
[2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[5 6 7 8 9]
ToTT
[1 2 3 5 7 9]
ToTT
[4 5 6 7 8 9]
ToTT
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7]
ToTT
[0 1 3 5 6 8]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[0 2 4 5 6 7]
[2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7]
ToTT
ToTT
[4 5 6 7 8 9]
[5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
[4 5 6 7 8]
ToTT
[1 2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[2 4 5 6 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
[2 3 4 5 6 7]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
[0 2 3 4 5 6]
ToTT
[1 2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
[2 3 5 6 7 8]
ToTT
ToTT
ToTT
ToTT
[4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5]
ToTT
ToTT
ToTT
[4 5 6 7 8 9]
[0 4 5 6 8 9]
ToTT
ToTT
[0 

ToTT
[3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7 8]
[0 1 2 5 6 9]
ToTT
ToTT
ToTT
[5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8]
[1 2 3 4 5 6]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
[5 6 7 8 9]
ToTT
ToTT
ToTT
[1 2 3 4 5 6 7]
ToTT
[0 2 4 5 6 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
ToTT
[4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7]
ToTT
[1 2 3 4 5 6 7 8 9]
[2 3 4 5 6 7 8 9]
ToTT
[5 6 7 8 9]
[1 2 3 4 5 8]
[0 1 2 3 4 5 6 7 8]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
[4 5 6 7 8 9]
ToTT
[0 3 5 6 7 9]
[2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 9]
ToTT
ToTT
ToTT
[1 2 3 4 5 6 7 8]
[4 5 6 7 8 9]
[1 2 4 5 7 9]
ToTT
[0 2 3 4 6 7]
ToTT
[3 4 5 6 7]
ToTT
[2 3 6 7 8 9]
[0 1 2 3 4 5 6]
[3 4 5 6 7 8 9]
ToTT
[0 1 2 5 7 8]
ToTT
ToTT
[3 4 5 6 7 8 9]
ToTT
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7]
ToTT
[0 2 3 4 6 8]
ToTT
[1 2 4 6 7 9]
ToTT
[1 2 3 4 5 6 

ToTT
[1 2 3 4 5 6 7]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7]
[3 4 5 6 7 8 9]
ToTT
[0 1 3 5 7 9]
ToTT
[1 2 3 4 5 6]
ToTT
[0 2 3 5 6 8]
ToTT
[1 2 3 4 5 9]
[5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
[0 1 5 7 8 9]
ToTT
ToTT
ToTT
ToTT
[2 3 4 5 6]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 4 6 8 9]
ToTT
ToTT
ToTT
ToTT
[0 1 2 3 4 5 6 7]
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7]
[0 1 3 6 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
ToTT
[1 2 3 4 5]
[1 2 3 4 5 6 7 8]
ToTT
[0 1 2 3 4 5]
ToTT
ToTT
[0 1 2 3 4 5 6 7]
ToTT
[2 3 4 5 6 7]
ToTT
[0 1 2 3 4 5 6]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
[5 6 7 8 9]
ToTT
[2 3 4 5 6]
ToTT
ToTT
ToTT
[4 5 6 7 8]
ToTT
[0 2 3 5 6 9]
ToTT
ToTT
[3 4 5 6 7]
ToTT
[3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8]
ToTT
[0 1 2 3 4 5 6 7 8]
ToTT
[0 2 4 6 8 9]
[1 2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[3 4 5 6 7 8]
ToTT
[3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT

ToTT
[1 2 3 4 5 6]
ToTT
[1 2 3 4 5 6 7 8 9]
[2 3 4 5 6 7 8 9]
ToTT
[4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
[2 3 4 5 6 7 8]
ToTT
ToTT
[0 1 3 4 6 7]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
[3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
[2 3 4 5 6 7 8 9]
ToTT
ToTT
[3 5 6 7 8 9]
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[0 2 4 5 6 7]
[0 1 2 3 4 5 6 7 8]
ToTT
[0 3 4 5 6 7]
ToTT
[0 2 4 6 7 8]
ToTT
ToTT
[2 3 4 5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
[0 3 5 6 7 9]
ToTT
ToTT
ToTT
[0 1 2 3 4 5 6 7]
ToTT
[0 1 4 5 7 9]
[1 2 3 4 5 6]
ToTT
[0 1 2 3 4 5 6 7]
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
[0 2 5 6 7 9]
[4 5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
ToTT
[2 3 4 5 6]
ToTT
[3 4 5 6 7 8 9]
[2 3 5 6 8 9]
ToTT
[0 2 4 5 6 7]
ToTT
ToTT
ToTT
[4 5 6 7 8]
[4 5 6 7 8 9]
ToTT
[0 1 2 3 4]
[1 2 3 4 5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
[4 5 6 7 8 9]
ToTT
[4 5 6 7

ToTT
[0 1 2 3 4 5]
ToTT
[5 6 7 8 9]
[0 1 2 3 4]
ToTT
ToTT
[0 1 4 6 7 9]
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4]
[0 1 2 7 8 9]
ToTT
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7]
ToTT
ToTT
ToTT
[0 1 2 3 4 5 6 7 8]
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7]
ToTT
ToTT
[1 2 3 5 6 8]
ToTT
ToTT
[0 1 3 4 5 9]
ToTT
ToTT
[0 1 2 3 5 6]
[2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 6 8]
ToTT
[4 5 6 7 8 9]
[3 4 5 6 7]
ToTT
ToTT
[3 4 5 6 7 8 9]
ToTT
ToTT
[2 3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
[1 2 3 4 5 6 7 8 9]
[1 2 3 4 5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
ToTT
[4 5 6 7 8 9]
ToTT
[0 1 2 3 5 7]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
ToTT
[5 6 7 8 9]
ToTT
[3 4 5 6 7 8 9]
ToTT
ToTT
ToTT
[5 6 7 8 9]
ToTT
ToTT
[5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
[2 4 5 6 7 8]
ToTT
[2 4 5 6 7 9]
[2 3 4 5 6 7 8 9]
ToTT
[1 2 3 4 5 6 7 8 9]
ToTT
[0 1 2 3 4 5 6 7 

In [ ]:
preds = learn.get_preds()

In [ ]:
(preds[0]>-0.5).sum()

tensor(2911)

In [ ]:
unweighted_profit(*preds, threshold=-0.5)

tensor(0.1795)

In [ ]:
for tfm in learn.dls[1].after_batch[2].tfms:
    tfm.split_idx=0

In [ ]:
learn.get_preds(dl=learn.dls[1])

(tensor([[-0.2676],
         [-0.8582],
         [-0.5350],
         ...,
         [-0.2764],
         [-0.6498],
         [-0.5167]]),
 tensor([ 100.0000,  -17.2117,  100.0000,  ...,  100.0000, -111.1111,
          100.0000]))

In [ ]:
tfms = [TSCrop, TSTimeNoise, TSDenoise]

In [ ]:
learn.randaugment(tfms)

AttributeError: 'Learner' object has no attribute 'data'

In [ ]:
learn.d

In [ ]:
preds = learn.get_preds(1)

In [ ]:
unweighted_profit(*preds, 0)

tensor(0.0724)

In [ ]:
(preds[0]>-0.5).sum()

tensor(9632)

In [ ]:
learn.loss_func(*preds)

tensor(-0.2047, requires_grad=True)

In [ ]:
weighted_profit(*preds, 0.5)

tensor(-0.2517)

In [ ]:
((preds[0].squeeze() > 0).float() * preds[1]).mean()

TensorFloat(-0.0200)

In [ ]:
((preds[0]>=-1).int() * preds[1]).mean()

TensorFloat(-2.8626)

In [ ]:
_verbose=False

In [ ]:
learn.fit_one_cycle(5, lr_max=1e-1, pct_start=0.3)

epoch,train_loss,valid_loss,unweighted_profit,time
0,-0.271601,-0.432503,-0.141485,00:03
1,-0.294538,-0.465206,-0.149836,00:03
2,-0.313153,-0.506633,-0.202532,00:03
3,-0.333494,-0.338696,-0.170355,00:03
4,-0.344131,-0.443630,-0.141831,00:03


timestepzero
randomcrop
crop
InceptionTimeSgm dtype torch.float32
torch.float32
block dtype torch.float32
[1 2 3 4 6 7 8 9]
timestepzero
InceptionTimeSgm dtype torch.float32
torch.float32
block dtype torch.float32
cutout
timestepzero
InceptionTimeSgm dtype torch.float32
torch.float32
block dtype torch.float32
[0 1 2 3 4 5]
randomcrop
crop
InceptionTimeSgm dtype torch.float32
torch.float32
block dtype torch.float32
timestepzero
maskout
InceptionTimeSgm dtype torch.float32
torch.float32
block dtype torch.float32
maskout
[3 4 5 6 7 8 9]
InceptionTimeSgm dtype torch.float32
torch.float32
block dtype torch.float32
dimout
timestepzero
InceptionTimeSgm dtype torch.float32
torch.float32
block dtype torch.float32
[2 3 4 5 6]
[1 2 3 4 5 6 7 8 9]
InceptionTimeSgm dtype torch.float32
torch.float32
block dtype torch.float32
timestepzero
[2 3 4 5 6 7 8 9]
InceptionTimeSgm dtype torch.float32
torch.float32
block dtype torch.float32
crop
maskout
InceptionTimeSgm dtype torch.float32
torch.float32
block

In [ ]:
x.data.shape

torch.Size([128, 6, 10])

In [ ]:
y.shape

torch.Size([128, 1])

In [ ]:
%%timeit
ToFloatTensor()(L(range(128)))

ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloa

ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloa

ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloa

ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloa

ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloa

ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloa

ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloa

ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloa

ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloatTensor
ToFloa

In [ ]:
%%timeit
ToTSTensor()(x.data)

ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor

ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor

ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor

ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor

ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor

ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor

ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor

ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor
ToTSTensor

In [ ]:
350*245 *1e-9

8.575000000000001e-05

In [ ]:
ToTSTensor(x.data)

RuntimeError: bool value of Tensor with more than one value is ambiguous

In [ ]:
ToTSTensor(x.)

RuntimeError: bool value of Tensor with more than one value is ambiguous